In [13]:
# call chmebl api with some basic molecule filters
from chembl_webresource_client.new_client import new_client
from tqdm import tqdm
import json
molecule = new_client.molecule
molecule.set_format('sdf')
# mols = molecule.filter(max_phase=4)
mols = molecule.filter(molecule_type="Small molecule").filter(molecule_properties__mw_freebase__lte=1000)
len(mols)

1899575

In [11]:
next(iter(mols)).decode('ascii')

'\n     RDKit          2D\n\n 24 26  0  0  0  0  0  0  0  0999 V2000\n    5.2792   -2.0500    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.7917   -2.3500    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0\n    5.2792   -1.4500    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0\n    6.3125   -2.0500    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0\n    5.7875   -4.7417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.7875   -4.1417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    6.3125   -1.4500    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    6.3042   -5.0417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.7917   -1.1500    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.7917   -2.9500    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.2667   -3.8417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    5.2667   -3.2417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    6.3042   -3.8417    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0\n    4.7

In [14]:
# write chembl query result to sdf file
with open('mols_chembl.sdf', 'w') as output:
      for mol in tqdm(mols):
          output.write(mol.decode('ascii'))
          output.write('$$$$\n')
# way too slow
# Error because of missing structure data

  0%|          | 525/1899575 [00:41<41:17:02, 12.78it/s]


HttpNotFound: Error for url https://www.ebi.ac.uk/chembl/api/data/molecule.sdf, server response: Molecule has no structure records.

In [6]:
import os
import time
from importlib import reload 
from rdkit import Chem
import pandas as pd
# data_dir = "../../data/Baltruschat/"
data_dir = "/data/shared/projects/pKa-prediction/pkasolver/data/Baltruschat/"
schroedinger_dir = "/data/shared/software/schrodinger2021-1/"
epik = schroedinger_dir + "epik"
convert = schroedinger_dir + "utilities/structconvert"

In [7]:
# convert .sdf file to .mae
os.system(f"{convert} {data_dir}novartis_cleaned_mono_unique_notraindata.sdf  {data_dir}nov.mae")
# predict pka of mols in .mae files with epik
os.system(f"{epik} -scan -imae {data_dir}nov.mae -omae {data_dir}nov_epik.mae")
# make the script wait until epik has finished
while os.path.isfile(f"{data_dir}nov_epik.mae") == False:
    time.sleep(2)
# convert epik output from .mae to .sdf
os.system(f"{convert} {data_dir}nov_epik.mae  {data_dir}nov_epik.sdf")

Converted file: /data/shared/projects/pKa-prediction/pkasolver/data/Baltruschat/nov.mae
JobId: a7srv2-0-617ba9df

Converted file: /data/shared/projects/pKa-prediction/pkasolver/data/Baltruschat/nov_epik.sdf


0

In [44]:
# unfinished script for loading in epik sdf
def load_sdf(input_file):
    suppl = Chem.SDMolSupplier(str(input_file), removeHs=False)
    df_rows = []
    for ind, mol in enumerate(suppl):
        if mol is not None:
            smiles = Chem.MolToSmiles(mol)
            df_rows.append([ind,smiles,mol])
            try:
                print(mol.GetPropsAsDict()['i_epik_pKa_atom_1'])
            except:
                pass
        if ind > 100:
            break
    mol_df = pd.DataFrame(df_rows, columns=('mol_id', 'smiles', 'mol'))
    return mol_df

In [45]:
load_sdf(f"{data_dir}nov_epik.sdf")

5
6
7
33


RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
RDKit WARNING: [14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero


,mol_id,smiles,mol
0,0,Brc1ccc(C2CN3C=CSC3=[NH+]2)cc1,<rdkit.Chem.rdchem.Mol object at 0x7f74eb57c640>
1,1,[H][n+]1ccc(C)cc1/C=C(\C)C1CC2OC2(C)CCCC(C)C(O...,<rdkit.Chem.rdchem.Mol object at 0x7f74d1172a60>
2,2,[H][n+]1cc(C)ccc1/C=C(\C)C1CC2OC2(C)CCCC(C)C(O...,<rdkit.Chem.rdchem.Mol object at 0x7f74cf1ccfa0>
3,3,[H][n+]1c(C)cccc1/C=C(\C)C1CC2OC2(C)CCCC(C)C(O...,<rdkit.Chem.rdchem.Mol object at 0x7f74d116eee0>
4,4,C/C(=C\c1ccccn1)C1CC2OC2(C)CCCC(C)C(O)C(C)C(=O...,<rdkit.Chem.rdchem.Mol object at 0x7f74d1172a00>
5,5,C1=C(c2ccc3ccccc3c2)N2CC[NH+]=C2c2ccccc21,<rdkit.Chem.rdchem.Mol object at 0x7f74d1172b20>
6,6,C1=C(c2cccc3ccccc23)N2CC[NH+]=C2c2ccccc21,<rdkit.Chem.rdchem.Mol object at 0x7f74cf15ea00>
7,7,C1=C(c2ccccc2)N2CC[NH+]=C2c2ccccc21,<rdkit.Chem.rdchem.Mol object at 0x7f74cf15e7c0>
8,8,C1=CN2CC(c3ccccc3)[NH+]=C2S1,<rdkit.Chem.rdchem.Mol object at 0x7f74d1172ca0>


[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:38:52] Warning: molecule is tagged as 3D, but all Z coords are zero
